# python连接Oracle数据库

In [2]:
import pandas as pd
import cx_Oracle
import os
os.environ['path']=r'D:\QlSql\instantclient_11_2'  #设置instantclient环境变量位置
connection=cx_Oracle.connect('sys','linzch2020','202.38.228.21:1521/ORCL',cx_Oracle.SYSDBA)#需要以SYSDBA身份进行登录，否则会报错

In [3]:
import pandas as pd
curs=connection.cursor()
sql='select * from HIGHWAY_MJTX_44_2020120208 where rownum<=2'
result=pd.read_sql(sql,connection)
result

,PASSID,GANTRYNO,GANTRYHEX,DIRECTION,GANTRYORDER,GANTRYTYPE,OPPOSITEGANTRYHEX,THROUGHTIME,MEDIATYPE,OBUID,...,ENSTATIONHEX,ENSTATIONNAME,ENTIME,ENSTATUS,ENWEIGHT,LASTGANTRYHEX,LASTGANTRYTIME,FEEMILES,DEALBEFORE,DEALAFTER
0,011101120123000289489320201202072531,G001544003013320030,4D0D71,2,203,1,4C0D71,2020-12-02 08:22:30,1,B1B1BEA911010001,...,44012602,广东海湾站,2020-12-02 07:25:31,1,7.0,4D0D0F,2020-12-02 08:12:17,191.0,79856.0,80047.0
1,011101120123000322511820201202081153,G001544003013320030,4D0D71,2,203,1,4C0D71,2020-12-02 08:39:31,1,B1B1BEA911010001,...,4401860B,广东两英站,2020-12-02 08:11:53,1,5.0,4D0D0F,2020-12-02 08:30:42,191.0,43072.0,43263.0


In [4]:
# 获取每条高速公路的roadid
sql='select roadname,roadid from tollstation2021 group by roadid,roadname'
pd1=pd.read_sql(sql,connection)
roadinfo=dict(zip(pd1['ROADNAME'],pd1['ROADID']))
roadinfo

{'新台高速': 'S0049440010',
 '梅河高速': 'G0025440030',
 '广惠高速': 'G0035440020',
 '京珠北': 'G0004440010',
 '阳阳高速': 'S0051440020',
 '清平一期': 'S0209440020',
 '机荷东': 'G0015440060',
 '增从高速': 'S0016440010',
 '广深高速': 'G0004440030',
 '惠盐深圳': 'G0015440050',
 '连英高速': 'G0078440070',
 '汕汾高速': 'G0015440010',
 '海湾大桥': 'G0015440020',
 '渝湛高速': 'G0075440010',
 '开阳高速': 'G0015440100',
 '黄埔大桥': 'G004W440010',
 '龙大高速': 'S0031440010',
 '盐坝高速': 'S0030440020',
 '普惠高速': 'S0017440020',
 '港珠澳大桥': 'G0094440200',
 '湛徐支线': 'G0015440140',
 '河惠莞': 'S0006440030',
 '潮漳高速': 'G1523440010',
 '潮惠高速': 'G15W3440010',
 '惠盐惠州': 'G0025440070',
 '广明高速': 'S0005440031',
 '从莞惠州': 'S0029440020',
 '沿海阳江': 'S0032440013',
 '机场高速': 'S0041440010',
 '惠深沿海': 'S0030440010',
 '东新高速': 'S0039440010',
 '广乐高速': 'G04W3440011',
 '水官高速': 'S0028440010',
 '深汕东': 'G0015440030',
 '韶赣高速': 'G6011440010',
 '虎门大桥': 'G9411440010',
 '江珠珠海段': 'S0047440012',
 '潮莞高速': 'G15W3440020',
 '北环高速': 'S0081440010',
 '西线一期': 'G0094440051',
 '湛徐高速': 'G0015440130',
 '梅观高速': 'G0094440

In [10]:
# 广佛高速为例，计算一天的货运量
day_list=['0'+str(x) for x in range(1,30) if x <10 ]#日期列表
day_list.extend([str(x) for x in range(10,31)])
id=roadinfo['广佛高速']#该路id
day='GDFTMJTX20210301'#表名，选择天数
per_weight={}
#获取当天每个车型的平均入口车重
for i in day_list:
    day='GDFTMJTX202106'+i
    print(day)
    s1='select vehicletype,avg(enweight) from '+ day + ' where substr(gantryno,1,11)='+'\''+id+'\''+' and vehicletype between 11 and 17 and \
    enweight!=0 group by vehicletype'#查询每种车型的平均重量
    print(s1)
    pd2=pd.read_sql(s1,connection)
    if i=='01':
        print(i)
        per_weight=dict(zip(pd2['VEHICLETYPE'],pd2['AVG(ENWEIGHT)']))
        print('per_weight')
        print(per_weight)
    else:
        temp_dict=dict(zip(pd2['VEHICLETYPE'],pd2['AVG(ENWEIGHT)']))
        print('temp_dict')
        print(temp_dict)
        for j in range(11,17):
            print(j)
            per_weight[j]+=temp_dict[j]

GDFTMJTX20210601
select vehicletype,avg(enweight) from GDFTMJTX20210601 where substr(gantryno,1,11)='S0015440010' and vehicletype between 11 and 17 and     enweight!=0 group by vehicletype
01
per_weight
{14: 22157.569204641946, 15: 25900.40268456376, 11: 4389.4422544661575, 12: 11003.320087495444, 13: 23961.97830298334, 16: 29914.854430379746}
GDFTMJTX20210602
select vehicletype,avg(enweight) from GDFTMJTX20210602 where substr(gantryno,1,11)='S0015440010' and vehicletype between 11 and 17 and     enweight!=0 group by vehicletype
temp_dict
{14: 20966.925382262998, 15: 22482.1768707483, 11: 5035.310136042358, 12: 10255.775267395746, 13: 21942.886844166012, 16: 28362.7868177137}
11
12
13
14
15
16
GDFTMJTX20210603
select vehicletype,avg(enweight) from GDFTMJTX20210603 where substr(gantryno,1,11)='S0015440010' and vehicletype between 11 and 17 and     enweight!=0 group by vehicletype
temp_dict
{14: 21828.76882022472, 15: 23578.115942028984, 11: 5958.235306565096, 12: 10748.716433814823, 13:

{14: 650754.7150601966,
 15: 738259.401373811,
 11: 193239.97681899153,
 12: 329595.9348090112,
 13: 734360.3827910451,
 16: 973140.8328932384}

In [26]:
per_weight#6月份30天的6种车型的总重值

{14: 650754.7150601966,
 15: 738259.401373811,
 11: 193239.97681899153,
 12: 329595.9348090112,
 13: 734360.3827910451,
 16: 973140.8328932384}

In [25]:
CarTypeWeight=[x/30 for x in list(per_weight.values())]
CarTypeWeight=dict(zip(list(per_weight.keys()),CarTypeWeight))


{14: 21691.823835339885,
 15: 24608.646712460366,
 11: 6441.332560633051,
 12: 10986.531160300372,
 13: 24478.67942636817,
 16: 32438.027763107948}

In [7]:
day_list=['0'+str(x) for x in range(1,30) if x <10 ]#日期列表
day_list.extend([str(x) for x in range(10,31)])
id=roadinfo['广佛高速']#该路id
day='GDFTMJTX202106'+day_list[0]#表名，选择天数
per_weight={}
s1='select vehicletype,avg(enweight) from '+ day + ' where substr(gantryno,1,11)='+'\''+id+'\''+' and vehicletype between 11 and 17 and \
    enweight!=0 group by vehicletype'#查询每种车型的平均重量
print(s1)
pd2=pd.read_sql(s1,connection)
pd2

select vehicletype,avg(enweight) from GDFTMJTX20210601 where substr(gantryno,1,11)='S0015440010' and vehicletype between 11 and 17 and     enweight!=0 group by vehicletype


,VEHICLETYPE,AVG(ENWEIGHT)
0,14,22157.569205
1,15,25900.402685
2,11,4389.442254
3,12,11003.320087
4,13,23961.978303
5,16,29914.854430
